# Einstieg in Keras

In [1]:
import numpy as np
import pandas as pd
import os
import PIL.Image as Image

from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Flatten, MaxPooling2D, Conv2D
import matplotlib.pyplot as plt

def getData(number_img, csv_path):
    '''
    Read images, and save to array
    :param number_img: the number of images to load
    :param csv_path: the path for CSV in which the image paths are stored
    :return: image_array, classes_array
    '''
    data = pd.read_csv(csv_path)

    image_paths = data['filename'][:number_img]

    #Create class vectors with following shape: [0,0,0,0,0,0,0,0,0,1] --> Represents class for number 9
    classes = data['label'][:number_img]
    classes_vecs = np.array([np.zeros(10) for classes in classes])
    for class_nr in range(len(classes)):
        classes_vecs[class_nr][classes[class_nr]] = 1

    images = []
    for i in range(len(image_paths)):
        path = os.path.join('data/Images/train', image_paths[i])
        img = Image.open(path).convert('RGBA')
        arr = np.array(img)
        images.append(arr)

    images = np.array(images)
    return images, classes_vecs

images, classes = getData(10000, "data/train.csv")

ModuleNotFoundError: No module named 'keras'

## 1 Das Model / Sequential
In Keras wird das Model des neuronalen Netzes Sequential genannt. Es stellt einen Stack aus Netzschichten dar. 
Zu einem Model können nach und nach die einzelnen Schichten via add Befehl hinzugefügt oder direkt im Konstruktor übergeben werden. 

Zum Beispiel: 

> Sequential([

>         Dense(64, input_dim=20, activation='relu'),
>         Dense(10, activation='softmax')
        
>     ])
    

> model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dense(10, activation='softmax'))

## 2 Layers
Übersicht hier: https://keras.io/layers/core/

### 2.1 Dense
Fully-Connected-Layer. 

> Dense(Anz_einheiten, activation=Aktivierungsfunktion)

> -->Dense(120, activation='relu')

### 2.2 Flatten
Übergang von z.B. Convolutional Layer zu Fully-Connected-Layer. 

Beispiel: 

> model.output_shape == (None, 64, 32, 32)

> model.add(Flatten())

> model.output_shape == (None, 65536)

### 2.3 Pooling
Pooling / Downsampling. 

> MaxPooling2D(pool_size=(pool_hoehe, pool_breite))
 
> -->MaxPooling2D(pool_size=(2, 2))

### 2.4 Convolutional
> Conv2D(Filter_anz, (Filter_hoehe, Filter_breite), activation=Aktivierungsfunktion, input_shape=Input_dimension)

> -->Conv2D(6, (9, 9), activation="relu", input_shape=images[0].shape)

Mit diesem Wissen können wir recht einfach ein Keras-Model für ein Convolutional Network erstellen: 

In [ ]:
modelK = Sequential([
        Conv2D(6, (9, 9), activation="relu", input_shape=images[0].shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(16, (9, 9), activation="relu", input_shape=images[0].shape),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(100, activation='relu'),
        Dense(10, activation='softmax')
    ])


## 3 Kompilierung des Models
Bevor das Model trainiert werden kann, muss es durch den Befehl compile kompiliert / konfiguriert werden. 
Der compile Befehl ist wie folgt aufgebaut: 

> model.compile(loss=Loss-Function,
>                  optimizer=Optimierer,
>                  metrics=Metric)

### 3.1 Loss
Hier alle in Keras verfügbaren Loss-Funktionen: 
https://keras.io/losses/

### 3.2 Optimierer
Optimierungsfunktionen, z.B. Stochastic Gradient Descent --> SGD

Alle verfügbaren Optimierungsfunktionen: 
https://keras.io/optimizers/

### 3.3 Metrics
Misst die Leistung des Netzes, z.B. accuracy

Alle Verfügbaren Metriken: 
https://keras.io/metrics/

Na dann kompolieren wir mal unser Model: 

In [ ]:
modelK.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

## Trainieren des Models
Nach der Kompilierung kann das Model angepasst / trainiert werden, dafür gibt es den fit Befehl. 

> model.fit(trainings_input, trainings_output, epochs=Trainingsdurchlaeufe, batch_size=Batchgroesse)

Das Trainieren unseres aktuellen Models mit vorher aufbereiteten Daten ist also ein Eizeiler: 

In [ ]:
model.fit(images, classes, epochs=20, batch_size=100)

## Vorhersagen
Letzendlich wollen wir natürlich unser Model testen und Vorhersagen machen. Dies geht in Keras ganz einfach mit dem Befehl predict
> model.predict(input_daten)

Schon wieder ein Einzeiler: 

In [ ]:
predictions = model.predict(np.array(test_images))